This is the final assigmnet for the capstone project for IBM Data Science Profesional Certification

As part of this project the fooloing activities will be performed:

    1. Shopping center data of both Paris, France and Millan, Italy are retrieved using Foursquare API
    2. To get the visual inspection, both the neighbours are plotted using Folium API
    3. Similarity of both city's neighborhood details w.r.t. shopping center are compared. Statistical average is used while making the comparison.
    4. Both the city's popuation within city limit are being considered.
    5. GDP per capita of France and Italy are compared.
    6. Based on the results obtained, shop opening destination for Smart Fashion Inc will be suggested.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # Install Folium API
import folium # map rendering library

print('All Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                              -                                /failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|4_73_pypy|5_73_pypy|3_73_pypy|2_73_pypy|1_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3

In [6]:
#Get Paris, France Geographical location Data
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address_paris = 'Paris, France'

geolocator_paris = Nominatim(user_agent="ny_explorer")
location_paris = geolocator_paris.geocode(address_paris)
latitude_paris = location_paris.latitude
longitude_paris = location_paris.longitude
print('The geograpical coordinate of Paris City are {}, {}.'.format(latitude_paris, longitude_paris))

The geograpical coordinate of Paris City are 48.8566969, 2.3514616.


In [61]:
#Get Millan, Italy Geographical location Data
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address_milan = 'Millan, Italy'

geolocator_milan = Nominatim(user_agent="ny_explorer")
location_milan = geolocator_millan.geocode(address_milan)
latitude_milan = location_milan.latitude
longitude_milan = location_milan.longitude
print('The geograpical coordinate of Milan City are {}, {}.'.format(latitude_milan, longitude_milan))

The geograpical coordinate of Milan City are 46.7071912, 11.6609414.


The city of Millan and Paris will be considered for prespective location.
To do this two different data sets are being compared.

1. The Population within the city limits of both the cities [Dataset available in: https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits]
2. The GDB of both the France & Italy [Dataset available in: https://en.wikipedia.org/wiki/Economy_of_the_European_Union ]

In [64]:
#Read the city limit Population data for Paris & Milan by web scrapping
url_city_population = 'https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits'
population_df_list = pd.read_html(url_city_population)

In [62]:
#Get the data in dataframe
population_df = population_df_list[0]


In [63]:

#Consider only Paris and Millan cities
options = ['Paris', 'Milan'] 
population_df_paris_milan = population_df[population_df['City'].isin(options)]
population_df_paris_milan

,City,Member State,Officialpopulation,Date of census,Reference,Photography
4,Paris,France,2140526,1 January 2019,[5][6],NaN
11,Milan,Italy,1404239,31 December 2019,[3],NaN


From the above comparison it is clear that the official city limit population in Paris is much higher than Milan.
More within City limit population signifies more buyers for the products.

Now we wil compare the GDP per capita of France and Italy. Country having more GDP per capita means people of that country has more money to specd and surely favorable as shopping and entertaintment hub.

In [65]:
#Web scrap Economy of the European Union page to get the GDP/per capita data
url_gdp = 'https://en.wikipedia.org/wiki/Economy_of_the_European_Union'
gdp_list = pd.read_html(url_gdp)

In [73]:
#Get the data in dataframe
gdp_df = gdp_list[4]
options_country = ['France', 'Italy'] 
gdp_paris_milan = gdp_df[gdp_df['Member state'].isin(options_country)]
gdp_paris_milan

,Member state,2012,2013,2014,2015,2016,2017,2018,2019
9,France,31820,32080,32420,33020,33430,34220,34980,35960
14,Italy,26920,26590,26770,27260,27970,28690,29220,29660


From the above comparison it is evidient that from 2012 to 2019 France gdp per capita is more than that of Italy.
This essentially makes France and its capital Paris as the favourable destination while opening a shop by Smart Fashion Inc.

Hence, we will consider Paris, France as the better location compared with Milan, Italy for the shop opening aspect.

Now we wil retrieve different neighborhood of Paris and do a comparison before giving final suggestion for location.

In [8]:
#Get the Neighborhood data of Paris and plot them in a Map
!wget -q -O 'paris_data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print('Paris Data downloaded!')

Paris Data downloaded!


In [22]:
with open('paris_data.json') as json_data:
    neighborhoods_data_paris = json.load(json_data)



In [79]:
# define the dataframe columns for Paris
column_names = ['nom_dept', 'population', 'postal_code', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods_paris = pd.DataFrame(columns=column_names)

In [80]:
#Populate the Paris Neighborhood Data
for data in neighborhoods_data_paris:
    neighborhood_name = data['fields']['nom_dept']
    population = data['fields']['population']
    postal_code =  data['fields']['postal_code']   
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods_paris = neighborhoods_paris.append({'nom_dept': neighborhood_name,
                                          'population': population,
                                           'postal_code':postal_code,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [81]:
#Print Paris Neighborhood data
neighborhoods_paris.head()

,nom_dept,population,postal_code,Latitude,Longitude
0,ESSONNE,15.5,91370,48.750443,2.251713
1,SEINE-ET-MARNE,0.2,77126,48.412561,3.052941
2,ESSONNE,0.3,91730,48.527268,2.197182
3,SEINE-ET-MARNE,20.2,77400,48.873070,2.709781
4,VAL-DE-MARNE,19.5,94110,48.805880,2.333510


In [83]:
#Create a map of Paris with neighborhoods superimposed on it
map_paris = folium.Map(location=[latitude_paris, longitude_paris], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_paris['Latitude'], neighborhoods_paris['Longitude'], neighborhoods_paris['nom_dept'], neighborhoods_paris['population']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris


Define Foursquare Credintials for data retrieval

In [96]:
CLIENT_ID = 'UJ4CK0WPFD01JRR3F1CTZCWVMVMU4OKY0CVDGKTZNUHLWCVO' 
CLIENT_SECRET = 'O1PQ1GTMXLCGN5PWUVAG3HXYUNR4KNAGHYZVLH3ZVTX3254A'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UJ4CK0WPFD01JRR3F1CTZCWVMVMU4OKY0CVDGKTZNUHLWCVO
CLIENT_SECRET:O1PQ1GTMXLCGN5PWUVAG3HXYUNR4KNAGHYZVLH3ZVTX3254A


Explore Neighborhood of Paris

In [33]:
#Get top 25 venues for each neignborhood within 500 meter radious
# type your answer here
LIMIT = 25 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [97]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The above function will be called to retrieve the neighborhood details of each of Neighborhood of Paris

In [98]:
# type your answer here
paris_neighborhood_venues = getNearbyVenues(names=neighborhoods_paris['nom_dept'],
                                   latitudes=neighborhoods_paris['Latitude'],
                                   longitudes=neighborhoods_paris['Longitude']
                                  )

ESSONNE
SEINE-ET-MARNE
ESSONNE
SEINE-ET-MARNE
VAL-DE-MARNE
SEINE-ET-MARNE
SEINE-ET-MARNE
SEINE-ET-MARNE
YVELINES
SEINE-ET-MARNE
ESSONNE
ESSONNE
VAL-D'OISE
SEINE-ET-MARNE
SEINE-ET-MARNE
SEINE-ET-MARNE
SEINE-ET-MARNE
ESSONNE
SEINE-ET-MARNE
ESSONNE
SEINE-ET-MARNE
VAL-D'OISE
VAL-D'OISE
YVELINES
ESSONNE
YVELINES
YVELINES
VAL-D'OISE
YVELINES
VAL-D'OISE
VAL-D'OISE
VAL-D'OISE
SEINE-ET-MARNE
SEINE-ET-MARNE
HAUTS-DE-SEINE
VAL-D'OISE
VAL-D'OISE
SEINE-ET-MARNE
YVELINES
YVELINES
ESSONNE
YVELINES
SEINE-ET-MARNE
VAL-DE-MARNE
YVELINES
YVELINES
SEINE-ET-MARNE
ESSONNE
SEINE-ET-MARNE
SEINE-ET-MARNE
SEINE-ET-MARNE
VAL-D'OISE
SEINE-ET-MARNE
SEINE-ET-MARNE
YVELINES
ESSONNE
SEINE-ET-MARNE
SEINE-ET-MARNE
VAL-D'OISE
SEINE-ET-MARNE
YVELINES
ESSONNE
YVELINES
ESSONNE
SEINE-ET-MARNE
YVELINES
SEINE-ET-MARNE
ESSONNE
VAL-D'OISE
YVELINES
SEINE-ET-MARNE
YVELINES
SEINE-ET-MARNE
VAL-D'OISE
SEINE-ET-MARNE
YVELINES
YVELINES
ESSONNE
ESSONNE
VAL-D'OISE
ESSONNE
VAL-D'OISE
YVELINES
SEINE-ET-MARNE
SEINE-ET-MARNE
VAL-DE-MARNE
SE

In [99]:
#Check the size of the resulting dataframe
print(paris_neighborhood_venues.shape)
paris_neighborhood_venues.head()

(3810, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ESSONNE,48.750443,2.251713,Poney Club de Verrières,48.747829,2.253170,Stables
1,ESSONNE,48.750443,2.251713,Restaurant Des Gatines,48.747892,2.249335,French Restaurant
2,VAL-DE-MARNE,48.805880,2.333510,"Arrêt Laplace RER [57,323,380,N21]",48.808075,2.332910,Bus Stop
3,VAL-DE-MARNE,48.805880,2.333510,Le Métro,48.808096,2.333146,Brasserie
4,VAL-DE-MARNE,48.805880,2.333510,Franprix,48.807761,2.331671,Grocery Store


In [100]:
#Now group-by the venues based on each neighborhood location
paris_neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ESSONNE,320,320,320,320,320,320
HAUTS-DE-SEINE,498,498,498,498,498,498
PARIS,1271,1271,1271,1271,1271,1271
SEINE-ET-MARNE,451,451,451,451,451,451
SEINE-SAINT-DENIS,216,216,216,216,216,216
VAL-D'OISE,308,308,308,308,308,308
VAL-DE-MARNE,351,351,351,351,351,351
YVELINES,395,395,395,395,395,395
